In [1]:
# Initialize
import numpy as np
import pandas as pd
import re

- 2 TMT-10plex kits were searched in the same PD study file
- Although the data is in one file, they must be treated as separate studies
    - P1 = 10plex containing CrAT KO (single knock out) and DKO (double knock out) with floxed controls
    - P2 = 10plex containing Sirt3 KO (single knock out) and DKO (double knock out) with floxed controls

In [2]:
peptides_file = 'pd22_exports/pd22_export_Williams_2019_PeptideGroups.txt'
proteins_file = 'pd22_exports/pd22_export_Williams_2019_Proteins.txt'
pep = pd.read_csv(peptides_file, sep='\t', low_memory=False)
pro = pd.read_csv(proteins_file, sep='\t', low_memory=False)

# Import data from proteome discoverer

For the peptides dataframe, use the first listed Master Protein Accession ID as the primary identifier. This identifier will be matched in the Proteins dataframe and used to pull additional information about the peptide. 

In [3]:
pep['Accession'] = pep['Master Protein Accessions'].str.split('; ', expand=True)[0]
pro['Entrez'] = pro['Entrez Gene ID'].str.split('; ', expand=True)[0]
pro['Ensembl'] = pro['Ensembl Gene ID'].str.split('; ', expand=True)[0]

pep = pep.merge(pro[['Accession', 'Description', 'Entrez', 'Gene Symbol']], on='Accession')

Next, import a pre-generated dataframe (stored as a pickled binary format). The information is based on the MitoCarta2 database, pulled from MitoMiner (http://mitominer.mrc-mbu.cam.ac.uk/release-4.0/begin.do) with additional, manually curated information about the particular location within the mitochondria. This will be used to identify peptides and proteins as mitochondrial.

IMS = intermembrane space

matrix = mitochondrial matrix

In [4]:
mitocarta_df = pd.read_csv('mitocarta.csv')

pep['MitoCarta2_List'] = pep.Entrez.isin(mitocarta_df.MouseGeneID.astype(str))
pep['Matrix'] = pep.Entrez.isin(mitocarta_df.MouseGeneID[mitocarta_df.Matrix].astype(str))
pep['IMS'] = pep.Entrez.isin(mitocarta_df.MouseGeneID[mitocarta_df.IMS].astype(str))

pro['MitoCarta2_List'] = pro.Entrez.isin(mitocarta_df.MouseGeneID.astype(str))
pro['Matrix'] = pro.Entrez.isin(mitocarta_df.MouseGeneID[mitocarta_df.Matrix].astype(str))
pro['IMS'] = pro.Entrez.isin(mitocarta_df.MouseGeneID[mitocarta_df.IMS].astype(str))

Next, we will identify columns that we want to store in a "Master Index". This will contian key information we want to retain from the proteome discoverer 2.2 exported file

In [5]:
# a list to of columns to make a master index
protein_mi_list = ['Accession', 
                   'Description',
                   'Entrez',
                   'Gene Symbol',
                   'Ensembl',
                   'MitoCarta2_List',
                   'Matrix', 
                   'IMS',
                   'Coverage [%]',
                   '# Peptides',
                   '# PSMs',
                   '# Protein Unique Peptides',
                   '# Unique Peptides', 
                   'Chromosome',
                   'Modifications',
                   'Acetyl (K) Count',
                   'Acetyl (K) Positions']

peptide_mi_list = ['Accession', 
                   'Description',
                   'Entrez',
                   'Gene Symbol',
                   'Modifications', 
                   'MitoCarta2_List',
                   'Matrix', 
                   'IMS', 
                   'Modifications in Proteins',
                   'Modifications in Proteins (all Sites)',
                   'Positions in Proteins',]

Now, we identify the abundance columns from the experiment. These contain the quantification data for each TMT channel. There are 2 fractions that are mentioned:

- F1 = "Input", the unenriched, pooled TMT-labeled peptides for all 10 samples. 
- F2 = "Acetyl", the Acetyl-lysine IP-enriched fraction 

The numbers (126, 130C) indicate the specific TMT tag associated with the column. 

In [6]:
p1_acetyl_abundance_list = ['Abundance: F1: 126, Sample, CrATvDKO plex1 (TMT Kit), Acetyl (Fraction), CrATfl_fl (Genotype)',
                            'Abundance: F1: 127N, Sample, CrATvDKO plex1 (TMT Kit), Acetyl (Fraction), CrATfl_fl (Genotype)',
                            'Abundance: F1: 127C, Sample, CrATvDKO plex1 (TMT Kit), Acetyl (Fraction), CrATfl_fl (Genotype)',
                            'Abundance: F1: 128N, Sample, CrATvDKO plex1 (TMT Kit), Acetyl (Fraction), CrATM_min_min (Genotype)',
                            'Abundance: F1: 128C, Sample, CrATvDKO plex1 (TMT Kit), Acetyl (Fraction), CrATM_min_min (Genotype)',
                            'Abundance: F1: 129N, Sample, CrATvDKO plex1 (TMT Kit), Acetyl (Fraction), CrATM_min_min (Genotype)',
                            'Abundance: F1: 129C, Sample, CrATvDKO plex1 (TMT Kit), Acetyl (Fraction), DFC (Genotype)',
                            'Abundance: F1: 130N, Sample, CrATvDKO plex1 (TMT Kit), Acetyl (Fraction), DFC (Genotype)',
                            'Abundance: F1: 130C, Sample, CrATvDKO plex1 (TMT Kit), Acetyl (Fraction), DKO (Genotype)',
                            'Abundance: F1: 131, Sample, CrATvDKO plex1 (TMT Kit), Acetyl (Fraction), DKO (Genotype)',]
                            
p2_acetyl_abundance_list = ['Abundance: F3: 126, Sample, S3vDKO plex2 (TMT Kit), Acetyl (Fraction), DFC (Genotype)',
                            'Abundance: F3: 127N, Sample, S3vDKO plex2 (TMT Kit), Acetyl (Fraction), DFC (Genotype)',
                            'Abundance: F3: 127C, Sample, S3vDKO plex2 (TMT Kit), Acetyl (Fraction), DKO (Genotype)',
                            'Abundance: F3: 128N, Sample, S3vDKO plex2 (TMT Kit), Acetyl (Fraction), DKO (Genotype)',
                            'Abundance: F3: 128C, Sample, S3vDKO plex2 (TMT Kit), Acetyl (Fraction), DKO (Genotype)',
                            'Abundance: F3: 129N, Sample, S3vDKO plex2 (TMT Kit), Acetyl (Fraction), Sirt3fl_fl (Genotype)',
                            'Abundance: F3: 129C, Sample, S3vDKO plex2 (TMT Kit), Acetyl (Fraction), Sirt3fl_fl (Genotype)',
                            'Abundance: F3: 130N, Sample, S3vDKO plex2 (TMT Kit), Acetyl (Fraction), Sirt3M_min_min (Genotype)',
                            'Abundance: F3: 130C, Sample, S3vDKO plex2 (TMT Kit), Acetyl (Fraction), Sirt3M_min_min (Genotype)',
                            'Abundance: F3: 131, Sample, S3vDKO plex2 (TMT Kit), Acetyl (Fraction), Sirt3M_min_min (Genotype)',]

These are shorter, more friendly versions of those identifier that will be used to rename the columns

Genotypes:
- S3FL = Sirt3 floxed control
- S3KO = Sirt3 knockout (via MCK-Cre)
- CrATFL = CrAT floxed control
- CrATKO = CrAT knockout (via MCK-Cre)
- DFC = Dual flox control (Sirt3 and CrAT floxed)
- DKO = Dual Knock-out (Sirt3 and CrAT knockout via MCK-Cre)

In [7]:
p1_acetyl_consolidated_list = ['CrATFL,126,1,Acetyl',
                               'CrATFL,127N,1,Acetyl',
                               'CrATFL,127C,1,Acetyl',
                               'CrATKO,128N,1,Acetyl',
                               'CrATKO,128C,1,Acetyl',
                               'CrATKO,129N,1,Acetyl',
                               'DFC,129C,1,Acetyl',
                               'DFC,130N,1,Acetyl',
                               'DKO,130C,1,Acetyl',
                               'DKO,131,1,Acetyl']
                                
p2_acetyl_consolidated_list = ['DFC,126,2,Acetyl',
                               'DFC,127N,2,Acetyl',
                               'DKO,127C,2,Acetyl',
                               'DKO,128N,2,Acetyl',
                               'DKO,128C,2,Acetyl',
                               'S3FL,129N,2,Acetyl',
                               'S3FL,129C,2,Acetyl',
                               'S3KO,130N,2,Acetyl',
                               'S3KO,130C,2,Acetyl',
                               'S3KO,131,2,Acetyl']

p1_input_consolidated_list = [x.replace('Acetyl', 'Input') for x in p1_acetyl_consolidated_list]
p2_input_consolidated_list = [x.replace('Acetyl', 'Input') for x in p2_acetyl_consolidated_list]

p1_input_abundance_list = [x.replace('Acetyl', 'Input').replace('F1', 'F2') for x in p1_acetyl_abundance_list]
p2_input_abundance_list = [x.replace('Acetyl', 'Input').replace('F3', 'F4') for x in p2_acetyl_abundance_list]

In [8]:
p1_acetyl_columns_df = pd.DataFrame.from_records([col.split(',') for col in p1_acetyl_consolidated_list])
p2_acetyl_columns_df = pd.DataFrame.from_records([col.split(',') for col in p2_acetyl_consolidated_list])

p1_input_columns_df = pd.DataFrame.from_records([col.split(',') for col in p1_input_consolidated_list])
p2_input_columns_df = pd.DataFrame.from_records([col.split(',') for col in p2_input_consolidated_list])

p1_acetyl_columns_df.set_index(p1_acetyl_columns_df.columns.tolist(), inplace=True)
p2_acetyl_columns_df.set_index(p2_acetyl_columns_df.columns.tolist(), inplace=True)

p1_input_columns_df.set_index(p1_input_columns_df.columns.tolist(), inplace=True)
p2_input_columns_df.set_index(p2_input_columns_df.columns.tolist(), inplace=True)

## Locate the Acetyl Peptides (Kac)

We look for peptides that were quantified from the Acetyl fraction AND contain an identified actyl-lysine residue present. While the acetyl fraction is enriched for acetylated lysine residues, not all identified peptides within this fraction contain this post-translational modification. 

In [9]:
# acetyl peptides
p1_acetyl_df = pep[pep['Modifications'].str.contains('Acetyl')][p1_acetyl_abundance_list].dropna()
p2_acetyl_df = pep[pep['Modifications'].str.contains('Acetyl')][p2_acetyl_abundance_list].dropna()

p1_acetyl_df.columns = p1_acetyl_columns_df.index
p2_acetyl_df.columns = p2_acetyl_columns_df.index

# Flatten the column headers for working in R
p1_acetyl_df.columns = ['_'.join(col).strip() for col in p1_acetyl_df.columns.values]
p2_acetyl_df.columns = ['_'.join(col).strip() for col in p2_acetyl_df.columns.values]

In [10]:
# Define the input - i.e., everything identified in the non-enriched fraction
p1_input_df = pep[p1_input_abundance_list].dropna()
p2_input_df = pep[p2_input_abundance_list].dropna()

p1_input_df.columns = p1_input_columns_df.index
p2_input_df.columns = p2_input_columns_df.index

# Flatten the column headers for working in R
p1_input_df.columns = ['_'.join(col).strip() for col in p1_input_df.columns.values]
p2_input_df.columns = ['_'.join(col).strip() for col in p2_input_df.columns.values]

## Locate the protein abundances
- Use values obtained from the input fraction - these represent the total proteome
- Only select proteins that are called "Master Proteins" by proteome discoverer 2.2 wiht a FDR < 0.01

In [11]:
p1_protein_df = (pro[(pro.Master=='IsMasterProtein')
                  &(pro['Exp. q-value: Combined']<=0.01)]
              [p1_input_abundance_list]
              .dropna(how='all')
              .copy())

p1_protein_df.columns = p1_input_df.columns
p1_protein_df.rename(columns=lambda x: x.replace('Input', 'Proteins'), inplace=True)

p2_protein_df = (pro[(pro.Master=='IsMasterProtein')
                  &(pro['Exp. q-value: Combined']<=0.01)]
              [p2_input_abundance_list]
              .dropna(how='all')
              .copy())

p2_protein_df.columns = p2_input_df.columns
p2_protein_df.rename(columns=lambda x: x.replace('Input', 'Proteins'), inplace=True)

# Normalize Data
- Correct for variations in total protein loading per kit
- use the mean of the sums of total peptide loading (input fraction) to generate a correction factor
- apply this correction factor to the acetyl peptides, 'input fraction', and proteins

## Calculate the scaling factor for each plex 

In [12]:
p1_channel_sums = p1_input_df.sum(axis=0)
p2_channel_sums = p2_input_df.sum(axis=0)

In [13]:
print('Plex 1: Individual Channel, Loading relative to the mean')

p1_channel_sums.divide(p1_channel_sums.mean())

Plex 1: Individual Channel, Loading relative to the mean


CrATFL_126_1_Input     0.796103
CrATFL_127N_1_Input    1.062434
CrATFL_127C_1_Input    1.102380
CrATKO_128N_1_Input    1.024336
CrATKO_128C_1_Input    0.995921
CrATKO_129N_1_Input    0.988315
DFC_129C_1_Input       1.038378
DFC_130N_1_Input       0.985814
DKO_130C_1_Input       0.956391
DKO_131_1_Input        1.049928
dtype: float64

In [14]:
print('Plex 1: Scaling factor to adjust relative loading to 1')

p1_scaling_factor = 1/p1_channel_sums.divide(p1_channel_sums.mean())
p1_scaling_factor

Plex 1: Scaling factor to adjust relative loading to 1


CrATFL_126_1_Input     1.256118
CrATFL_127N_1_Input    0.941235
CrATFL_127C_1_Input    0.907128
CrATKO_128N_1_Input    0.976243
CrATKO_128C_1_Input    1.004096
CrATKO_129N_1_Input    1.011824
DFC_129C_1_Input       0.963041
DFC_130N_1_Input       1.014390
DKO_130C_1_Input       1.045597
DKO_131_1_Input        0.952446
dtype: float64

In [15]:
print('Plex 2: Individual Channel, Loading relative to the mean')

p2_channel_sums.divide(p2_channel_sums.mean())

Plex 2: Individual Channel, Loading relative to the mean


DFC_126_2_Input      0.928235
DFC_127N_2_Input     1.083377
DKO_127C_2_Input     1.100762
DKO_128N_2_Input     1.069166
DKO_128C_2_Input     0.973270
S3FL_129N_2_Input    0.950931
S3FL_129C_2_Input    1.034328
S3KO_130N_2_Input    0.958474
S3KO_130C_2_Input    0.981884
S3KO_131_2_Input     0.919571
dtype: float64

In [16]:
print('Plex 2: Scaling factor to adjust relative loading to 1')

p2_scaling_factor = 1/p2_channel_sums.divide(p2_channel_sums.mean())
p2_scaling_factor

Plex 2: Scaling factor to adjust relative loading to 1


DFC_126_2_Input      1.077313
DFC_127N_2_Input     0.923039
DKO_127C_2_Input     0.908461
DKO_128N_2_Input     0.935309
DKO_128C_2_Input     1.027465
S3FL_129N_2_Input    1.051601
S3FL_129C_2_Input    0.966811
S3KO_130N_2_Input    1.043325
S3KO_130C_2_Input    1.018450
S3KO_131_2_Input     1.087463
dtype: float64

## Apply the Scaling Factor

In [17]:
p1_input_norm = (p1_input_df * p1_scaling_factor)
p1_input_norm.columns = [str(col) + '_norm' for col in p1_input_norm.columns]
p1_input_norm.sum()

CrATFL_126_1_Input_norm     3029072.32
CrATFL_127N_1_Input_norm    3029072.32
CrATFL_127C_1_Input_norm    3029072.32
CrATKO_128N_1_Input_norm    3029072.32
CrATKO_128C_1_Input_norm    3029072.32
CrATKO_129N_1_Input_norm    3029072.32
DFC_129C_1_Input_norm       3029072.32
DFC_130N_1_Input_norm       3029072.32
DKO_130C_1_Input_norm       3029072.32
DKO_131_1_Input_norm        3029072.32
dtype: float64

In [18]:
p2_input_norm = (p2_input_df * p2_scaling_factor)
p2_input_norm.columns = [str(col) + '_norm' for col in p2_input_norm.columns]
p2_input_norm.sum()

DFC_126_2_Input_norm      2835985.02
DFC_127N_2_Input_norm     2835985.02
DKO_127C_2_Input_norm     2835985.02
DKO_128N_2_Input_norm     2835985.02
DKO_128C_2_Input_norm     2835985.02
S3FL_129N_2_Input_norm    2835985.02
S3FL_129C_2_Input_norm    2835985.02
S3KO_130N_2_Input_norm    2835985.02
S3KO_130C_2_Input_norm    2835985.02
S3KO_131_2_Input_norm     2835985.02
dtype: float64

In [19]:
p1_protein_norm = (p1_protein_df * p1_scaling_factor.rename(lambda x: x.replace('Input', 'Proteins')))
p1_protein_norm.columns = [str(col) + '_norm' for col in p1_protein_norm.columns]
p1_protein_norm.sum()

CrATFL_126_1_Proteins_norm     1.801427e+06
CrATFL_127N_1_Proteins_norm    1.812383e+06
CrATFL_127C_1_Proteins_norm    1.805524e+06
CrATKO_128N_1_Proteins_norm    1.777052e+06
CrATKO_128C_1_Proteins_norm    1.813146e+06
CrATKO_129N_1_Proteins_norm    1.783128e+06
DFC_129C_1_Proteins_norm       1.812656e+06
DFC_130N_1_Proteins_norm       1.785270e+06
DKO_130C_1_Proteins_norm       1.806918e+06
DKO_131_1_Proteins_norm        1.797056e+06
dtype: float64

In [20]:
p2_protein_norm = (p2_protein_df * p2_scaling_factor.rename(lambda x: x.replace('Input', 'Proteins')))
p2_protein_norm.columns = [str(col) + '_norm' for col in p2_protein_norm.columns]
p2_protein_norm.sum()

DFC_126_2_Proteins_norm      1.654504e+06
DFC_127N_2_Proteins_norm     1.663691e+06
DKO_127C_2_Proteins_norm     1.692797e+06
DKO_128N_2_Proteins_norm     1.658630e+06
DKO_128C_2_Proteins_norm     1.697719e+06
S3FL_129N_2_Proteins_norm    1.656868e+06
S3FL_129C_2_Proteins_norm    1.711811e+06
S3KO_130N_2_Proteins_norm    1.661136e+06
S3KO_130C_2_Proteins_norm    1.649256e+06
S3KO_131_2_Proteins_norm     1.657424e+06
dtype: float64

The intensities for the acetyl peptides will not sum to the same number after this correction factor - this is because there are different levels of acetylation in the genotypes. You can see that the Genetic knock-outs have ↑ signal relative their flox control

In [21]:
p1_acetyl_norm = p1_acetyl_df * p1_scaling_factor.rename(lambda x: x.replace('Input', 'Acetyl'))
p1_acetyl_norm.columns = [str(col) + '_norm' for col in p1_acetyl_norm.columns]
p1_acetyl_norm.sum()

CrATFL_126_1_Acetyl_norm      82407.140456
CrATFL_127N_1_Acetyl_norm     76833.319507
CrATFL_127C_1_Acetyl_norm     80878.155561
CrATKO_128N_1_Acetyl_norm     97116.707060
CrATKO_128C_1_Acetyl_norm    101603.344537
CrATKO_129N_1_Acetyl_norm     93492.503420
DFC_129C_1_Acetyl_norm        83985.925779
DFC_130N_1_Acetyl_norm        77553.166664
DKO_130C_1_Acetyl_norm       153635.212301
DKO_131_1_Acetyl_norm        136605.685611
dtype: float64

In [22]:
p2_acetyl_norm = p2_acetyl_df * p2_scaling_factor.rename(lambda x: x.replace('Input', 'Acetyl'))
p2_acetyl_norm.columns = [str(col) + '_norm' for col in p2_acetyl_norm.columns]
p2_acetyl_norm.sum()

DFC_126_2_Acetyl_norm       50134.603739
DFC_127N_2_Acetyl_norm      51292.740622
DKO_127C_2_Acetyl_norm     127262.067606
DKO_128N_2_Acetyl_norm     105069.564430
DKO_128C_2_Acetyl_norm     115084.761966
S3FL_129N_2_Acetyl_norm     42498.438985
S3FL_129C_2_Acetyl_norm     51771.657244
S3KO_130N_2_Acetyl_norm     55096.749688
S3KO_130C_2_Acetyl_norm     57722.696542
S3KO_131_2_Acetyl_norm      52647.349156
dtype: float64

# Calculate the "relative occupancy" for each plex
- This adjusts the acetyl peptide quantification based on changes in protein quantification. This corrects not for experimental loading, but for genetic changes that may occur between the genetic knock-out and the flox control.
- The relative occupancy corrections will be almost un-noticed in these animals, however. 

In [23]:
# convert the input-normalized data into the log2 space
# subtract the row mean from each column to center each row around 0 (in log 2)
p1_acetyl_norm_log2   = np.log2(p1_acetyl_norm).sub(np.log2(p1_acetyl_norm).mean(axis=1), axis='index')
p1_input_norm_log2    = np.log2(p1_input_norm).sub(np.log2(p1_input_norm).mean(axis=1), axis='index')
p1_protein_norm_log2  = np.log2(p1_protein_norm).sub(np.log2(p1_protein_norm).mean(axis=1), axis='index')

p2_acetyl_norm_log2   = np.log2(p2_acetyl_norm).sub(np.log2(p2_acetyl_norm).mean(axis=1), axis='index')
p2_input_norm_log2    = np.log2(p2_input_norm).sub(np.log2(p2_input_norm).mean(axis=1), axis='index')
p2_protein_norm_log2  = np.log2(p2_protein_norm).sub(np.log2(p2_protein_norm).mean(axis=1), axis='index')


# rename columns to save the information
p1_acetyl_norm_log2.columns  = [str(col) + '_log2' for col in p1_acetyl_norm_log2.columns]
p1_input_norm_log2.columns   = [str(col) + '_log2' for col in p1_input_norm_log2.columns]
p1_protein_norm_log2.columns = [str(col) + '_log2' for col in p1_protein_norm_log2.columns]

p2_acetyl_norm_log2.columns  = [str(col) + '_log2' for col in p2_acetyl_norm_log2.columns]
p2_input_norm_log2.columns   = [str(col) + '_log2' for col in p2_input_norm_log2.columns]
p2_protein_norm_log2.columns = [str(col) + '_log2' for col in p2_protein_norm_log2.columns]

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log2
  import sys


Combine the the "raw", load-normalized, and log2-transformed data into a single dataframe

In [24]:
# plex 1
p1_acetyl = pd.concat([pep.loc[p1_acetyl_norm.index][peptide_mi_list], 
                       p1_acetyl_df, 
                       p1_acetyl_norm, 
                       p1_acetyl_norm_log2,], 
                      axis=1)

p1_input = pd.concat([pep.loc[p1_input_norm.index][peptide_mi_list], 
                      p1_input_df, 
                      p1_input_norm, 
                      p1_input_norm_log2], 
                     axis=1)

p1_protein = pd.concat([pro.loc[p1_protein_df.index][protein_mi_list], 
                        p1_protein_df, 
                        p1_protein_norm, 
                        p1_protein_norm_log2], 
                       axis=1)

# plex2
p2_acetyl = pd.concat([pep.loc[p2_acetyl_norm.index][peptide_mi_list], 
                       p2_acetyl_df, 
                       p2_acetyl_norm, 
                       p2_acetyl_norm_log2,], 
                      axis=1)

p2_input = pd.concat([pep.loc[p2_input_norm.index][peptide_mi_list], 
                      p2_input_df, 
                      p2_input_norm, 
                      p2_input_norm_log2], 
                     axis=1)

p2_protein = pd.concat([pro.loc[p2_protein_df.index][protein_mi_list], 
                        p2_protein_df, 
                        p2_protein_norm, 
                        p2_protein_norm_log2], 
                       axis=1)

## Calculate the RO
- Merge protein information along with acetyl information into a single dataframe
- The protein abundance information will be used to correc the acetyl data (the relative occupancy)

In [25]:
p1_protein_merge_cols = p1_protein.filter(regex='_log2').columns.tolist() + ['Accession']
p2_protein_merge_cols = p2_protein.filter(regex='_log2').columns.tolist() + ['Accession']

In [26]:
p1_merged_acetyl = p1_acetyl.merge(p1_protein[p1_protein_merge_cols], 
                                   on='Accession', 
                                   how='left', 
                                   validate='m:1')

p2_merged_acetyl = p2_acetyl.merge(p2_protein[p2_protein_merge_cols], 
                                   on='Accession', 
                                   how='left', 
                                   validate='m:1')

The relative occupancy of the post-translational modification is calculated by subtracting changes in the protien abundance from the acetyl-peptide abundance. For example, if a PTM site is found to be increasing by 2 fold, but the protein abundnace is also increasing by 2 fold, then the relative occupancy would be 0. 

In [28]:
# calculate relative occupancy in acetyl peptides
# p1
p1_relative_occupancy = (p1_merged_acetyl.filter(regex='Acetyl_norm_log2')
                         .sub(p1_merged_acetyl.filter(regex='Proteins_norm_log2')
                              # rename the protein column headers to enable easy dataframe subtraction
                              .rename(columns=lambda x: x.replace('Proteins', 'Acetyl'))))

p1_relative_occupancy.rename(columns=lambda x: x+'_ro', inplace=True)

p1_relative_occupancy.head()

,CrATFL_126_1_Acetyl_norm_log2_ro,CrATFL_127N_1_Acetyl_norm_log2_ro,CrATFL_127C_1_Acetyl_norm_log2_ro,CrATKO_128N_1_Acetyl_norm_log2_ro,CrATKO_128C_1_Acetyl_norm_log2_ro,CrATKO_129N_1_Acetyl_norm_log2_ro,DFC_129C_1_Acetyl_norm_log2_ro,DFC_130N_1_Acetyl_norm_log2_ro,DKO_130C_1_Acetyl_norm_log2_ro,DKO_131_1_Acetyl_norm_log2_ro
0,-0.228927,-0.397147,-0.430871,-0.171143,-0.068982,0.235369,-0.013883,0.582442,0.045253,0.447889
1,-0.064612,-0.413235,-0.386901,-0.077529,-0.448411,0.235798,-0.194681,0.441240,0.466737,0.441594
2,-0.500649,-0.527844,-0.044728,0.013550,-0.424221,0.130864,-0.369122,0.680328,0.784873,0.256949
3,-0.142318,-0.359298,-0.391600,0.082026,-0.261988,0.064649,-0.338449,0.675071,0.197805,0.474103
4,-0.239486,-0.607886,0.017914,0.547402,-0.433459,0.139333,0.155283,0.713918,-0.149299,-0.143720


In [29]:
# calculate relative occupancy in acetyl peptides

p2_relative_occupancy = (p2_merged_acetyl.filter(regex='Acetyl_norm_log2')
                         .sub(p2_merged_acetyl.filter(regex='Proteins_norm_log2')
                              # rename the protein column headers to enable easy dataframe subtraction
                              .rename(columns=lambda x: x.replace('Proteins', 'Acetyl'))))

p2_relative_occupancy.rename(columns=lambda x: x+'_ro', inplace=True)

p2_relative_occupancy.head()

,DFC_126_2_Acetyl_norm_log2_ro,DFC_127N_2_Acetyl_norm_log2_ro,DKO_127C_2_Acetyl_norm_log2_ro,DKO_128N_2_Acetyl_norm_log2_ro,DKO_128C_2_Acetyl_norm_log2_ro,S3FL_129N_2_Acetyl_norm_log2_ro,S3FL_129C_2_Acetyl_norm_log2_ro,S3KO_130N_2_Acetyl_norm_log2_ro,S3KO_130C_2_Acetyl_norm_log2_ro,S3KO_131_2_Acetyl_norm_log2_ro
0,-1.020246,-0.372482,1.015874,1.300822,1.238225,-1.448226,-0.824477,0.024452,0.108021,-0.021963
1,1.197794,0.466177,-0.275599,0.123803,0.143688,0.413960,-1.323232,-0.263887,-0.061468,-0.421236
2,-0.073142,-0.065819,-0.016426,0.267592,0.051071,0.005400,-1.184937,0.363596,0.602540,0.050127
3,0.661939,0.421147,-0.106358,0.367535,0.294433,0.110343,-1.364902,-0.480644,0.669652,-0.573143
4,0.753240,0.693138,-0.001591,1.022336,-0.454821,-0.389432,-0.410803,-0.208252,-0.410613,-0.593201


In [30]:
# stick the relative occupancy data onto the merged data
p1_merged_acetyl = pd.concat([p1_merged_acetyl, p1_relative_occupancy], axis=1)
p2_merged_acetyl = pd.concat([p2_merged_acetyl, p2_relative_occupancy], axis=1)

In [31]:
# drop the protein columns out of the data
p1_merged_acetyl.drop(columns=p1_merged_acetyl.filter(regex='_Protein').columns.tolist(), 
                      inplace=True)

p2_merged_acetyl.drop(columns=p2_merged_acetyl.filter(regex='_Protein').columns.tolist(), 
                      inplace=True)

# *limma* analysis

## Export Files for *limma* analysis

It is probably just as easy to do this entire analysis in R, with the limma component incorporated into the data processing. I'm not very experienced in R, so the disjointed analysis with R and python was faster for me. Exporting the data from Python into a CSV to import into R was the quickest method for my purposes. One could also try the rpy2 library (I tinkered with it, but found it to be too much effort for the payoff) or exporting the dataframes from pandas into a feather/arrow format.

In [32]:
p1_acetyl_norm.to_csv('limma/p1_limma_export_kac.csv')
p2_acetyl_norm.to_csv('limma/p2_limma_export_kac.csv')

p1_merged_acetyl.filter(regex='_ro$').to_csv('limma/p1_limma_export_RO_kac.csv')
p2_merged_acetyl.filter(regex='_ro$').to_csv('limma/p2_limma_export_RO_kac.csv')

p1_protein.filter(regex='.+_norm$').to_csv('limma/p1_limma_export_pro.csv')
p2_protein.filter(regex='.+_norm$').to_csv('limma/p2_limma_export_pro.csv')

## Read Files from *limma* analysis

In [33]:
p1_kac_limma = pd.read_csv('limma/p1_eb_fit_kac.tsv', sep='\t')
p2_kac_limma = pd.read_csv('limma/p2_eb_fit_kac.tsv', sep='\t')

p1_kac_ro_limma = pd.read_csv('limma/p1_eb_fit_RO_kac.tsv', sep='\t')
p2_kac_ro_limma = pd.read_csv('limma/p2_eb_fit_RO_kac.tsv', sep='\t')

p1_prot_limma = pd.read_csv('limma/p1_eb_fit_pro.tsv', sep='\t')
p2_prot_limma = pd.read_csv('limma/p2_eb_fit_pro.tsv', sep='\t')

In [34]:
# merge back onto data
p1_acetyl = pd.concat([p1_acetyl, p1_kac_limma], axis=1)
p2_acetyl = pd.concat([p2_acetyl, p2_kac_limma], axis=1)

p1_protein = pd.concat([p1_protein, p1_prot_limma], axis=1)
p2_protein = pd.concat([p2_protein, p2_prot_limma], axis=1)

p1_ro_acetyl = pd.concat([p1_merged_acetyl, p1_kac_ro_limma], axis=1)
p2_ro_acetyl = pd.concat([p2_merged_acetyl, p2_kac_ro_limma], axis=1)

Rename the limma results to have more intuitive names

- `Coef` = Log2 Fold Change comparison between two groups
- `Res` = Significance. 
    - 0 = not significant
    - 1 = significant difference increasing
    - -1 = significant difference decreasing

In [35]:
limma_match = ['Coef\.', 'p\.value\.', 't\.', 'Res\.']
limma_replace = ['_Log2FC', '_p_value', '_t', '_significant']

for match, replace in zip(limma_match, limma_replace):
    p1_acetyl.rename(columns=lambda x: re.sub(match + '(.+)', '\g<1>' + replace, x), inplace=True)
    p1_protein.rename(columns=lambda x: re.sub(match + '(.+)', '\g<1>' + replace, x), inplace=True)
    p1_ro_acetyl.rename(columns=lambda x: re.sub(match + '(.+)', '\g<1>' + replace, x), inplace=True)
    
    p2_acetyl.rename(columns=lambda x: re.sub(match + '(.+)', '\g<1>' + replace, x), inplace=True)
    p2_protein.rename(columns=lambda x: re.sub(match + '(.+)', '\g<1>' + replace, x), inplace=True)
    p2_ro_acetyl.rename(columns=lambda x: re.sub(match + '(.+)', '\g<1>' + replace, x), inplace=True)

Pull out useful information from the Description field into separate columns. Although there is a Gene Symbol field present, it isn't always populated and isn't always the same

In [36]:
p1_acetyl['Name'] = p1_acetyl.Description.str.split(' [OSGNPESV]{2}=', expand=True)[0]
p1_acetyl['Symbol'] = p1_acetyl.Description.str.split(' [OSGNPESV]{2}=', expand=True)[2]

p1_protein['Name'] = p1_protein.Description.str.split(' [OSGNPESV]{2}=', expand=True)[0]
p1_protein['Symbol'] = p1_protein.Description.str.split(' [OSGNPESV]{2}=', expand=True)[2]

p1_ro_acetyl['Name'] = p1_ro_acetyl.Description.str.split(' [OSGNPESV]{2}=', expand=True)[0]
p1_ro_acetyl['Symbol'] = p1_ro_acetyl.Description.str.split(' [OSGNPESV]{2}=', expand=True)[2]

# reorder the columns so the information
p1_acetyl = p1_acetyl[p1_acetyl.columns.tolist()[-2:] + p1_acetyl.columns.tolist()[:-2]]
p1_protein = p1_protein[p1_protein.columns.tolist()[-2:] + p1_protein.columns.tolist()[:-2]]
p1_ro_acetyl = p1_ro_acetyl[p1_ro_acetyl.columns.tolist()[-2:] + p1_ro_acetyl.columns.tolist()[:-2]]

In [37]:
p2_acetyl['Name'] = p2_acetyl.Description.str.split(' [OSGNPESV]{2}=', expand=True)[0]
p2_acetyl['Symbol'] = p2_acetyl.Description.str.split(' [OSGNPESV]{2}=', expand=True)[2]

p2_protein['Name'] = p2_protein.Description.str.split(' [OSGNPESV]{2}=', expand=True)[0]
p2_protein['Symbol'] = p2_protein.Description.str.split(' [OSGNPESV]{2}=', expand=True)[2]

p2_ro_acetyl['Name'] = p2_ro_acetyl.Description.str.split(' [OSGNPESV]{2}=', expand=True)[0]
p2_ro_acetyl['Symbol'] = p2_ro_acetyl.Description.str.split(' [OSGNPESV]{2}=', expand=True)[2]

# reorder the columns so the information
p2_acetyl = p2_acetyl[p2_acetyl.columns.tolist()[-2:] + p2_acetyl.columns.tolist()[:-2]]
p2_protein = p2_protein[p2_protein.columns.tolist()[-2:] + p2_protein.columns.tolist()[:-2]]
p2_ro_acetyl = p2_ro_acetyl[p2_ro_acetyl.columns.tolist()[-2:] + p2_ro_acetyl.columns.tolist()[:-2]]

Sort the data

In [38]:
p1_protein.sort_values('Acetyl (K) Count', ascending=False, inplace=True)
p2_protein.sort_values('Acetyl (K) Count', ascending=False, inplace=True)

p1_acetyl.sort_values('DKOvsDFC_Log2FC', ascending=False, inplace=True)
p2_acetyl.sort_values('DKOvsDFC_Log2FC', ascending=False, inplace=True)

p1_ro_acetyl.sort_values('DKOvsDFC_Log2FC', ascending=False, inplace=True)
p2_ro_acetyl.sort_values('DKOvsDFC_Log2FC', ascending=False, inplace=True)

# Top 25 Fold Changes

This is one (simple) method to assess the severity of hyperacetylation. We'll only look at the relative occupancy corrected acetylpeptide abundances for this analysis

First, we will limit our analysis to the mitochondria as defined by the mitocarta2 dataset

In [39]:
p1 = p1_ro_acetyl[p1_ro_acetyl.MitoCarta2_List==True].copy()
p2 = p2_ro_acetyl[p2_ro_acetyl.MitoCarta2_List==True].copy()

Now, we will move out of the log2 space for more intuitive comparisons

In [41]:
p1['DKOvsDFC_FC'] = np.exp2(p1.DKOvsDFC_Log2FC)
p1['CrATKOvsCrATFL_FC'] = np.exp2(p1.CrATKOvsCrATFL_Log2FC)

p2['DKOvsDFC_FC'] = np.exp2(p2.DKOvsDFC_Log2FC)
p2['S3KOvsS3FL_FC'] = np.exp2(p2.S3KOvsS3FL_Log2FC)

Now, we want to pull out the specific lysine residues acetylated on each individual peptide. We want these residue positions related back to the master protein (i.e., not the position within the peptide). This isn't easily achieved in how the PD data is currently exported. Some peptides could correspond to multiple protein accessions, each with different lysine positions. All possible accessions and positions are listed in the peptide output - however, the order is arbitrary. Therefor, we need to iterate though the accessions listed and match them back to the master protein accession. 

In [42]:
p1_kac_sites = p1['Modifications in Proteins'].str.split('; (?!K)', expand=True, )
p1_kac_sites.head()

,0,1,2,3,4,5,6,7,8,9
919,Q61425 1xAcetyl [K312],None,None,None,None,None,None,None,None,None
1143,Q9CZU6 1xAcetyl [K393],None,None,None,None,None,None,None,None,None
764,Q8K2B3 1xAcetyl [K179],A0A1Y7VJ55 1xAcetyl [K177],None,None,None,None,None,None,None,None
550,Q8BMS1 1xAcetyl [K334],None,None,None,None,None,None,None,None,None
1145,Q9CZU6 1xAcetyl [K52],None,None,None,None,None,None,None,None,None


In [50]:
p2_kac_sites = p2['Modifications in Proteins'].str.split('; (?!K)', expand=True, )
p2_kac_sites.head(10)

,0,1,2,3,4,5,6,7,8,9,10
752,Q9CZU6 1xAcetyl [K52],None,None,None,None,None,None,None,None,None,None
661,Q8BWT1 1xAcetyl [K209],None,None,None,None,None,None,None,None,None,None
750,Q9CZU6 1xAcetyl [K393],None,None,None,None,None,None,None,None,None,None
609,Q9WUR2 1xAcetyl [K60],E9PUY9 1xAcetyl [K27],E9PVM6 1xAcetyl [K27],Q3TCD4 1xAcetyl [K47],E9PYA9 1xAcetyl [K27],E9Q858 1xAcetyl [K27],Q9WUR2-2 1xAcetyl [K27],E9PYC6 1xAcetyl [K27],E9Q7A8 1xAcetyl [K27],None,None
341,P08249 1xAcetyl [K239],None,None,None,None,None,None,None,None,None,None
280,Q8BMS1 1xAcetyl [K259],None,None,None,None,None,None,None,None,None,None
812,P03930 1xAcetyl [K46],None,None,None,None,None,None,None,None,None,None
301,Q8BMS1 2xAcetyl [K411; K413],None,None,None,None,None,None,None,None,None,None
286,Q8BMS1 1xAcetyl [K],None,None,None,None,None,None,None,None,None,None
768,F7D3P8 1xAcetyl [K31],Q9DB20 1xAcetyl [K97],None,None,None,None,None,None,None,None,None


In the above data, peptides with multiple accessions can have the master protein accession listed in any of the 10 columns above - it isn't always in the first column. Therefore, we will iterate through the columns of these sites to match them back to the original accession. The matches will be combined into a list, and the list will be transformed into a Series

In [47]:
# list to hold the sites associated with the same master protein accession
p1_sites = []
for i in np.arange(0, p1_kac_sites.columns[-1]+1):
    bool_mask = p1.Accession == p1_kac_sites[i].str.split(' ', expand=True)[0]
    try:
        temp_sites = p1_kac_sites[i].loc[bool_mask].str.split('(?<!;)\s', expand=True)[2]
        p1_sites.append(temp_sites)
    except:
        continue
        
p2_sites = []
for i in np.arange(0, p2_kac_sites.columns[-1]+1):
    bool_mask = p2.Accession == p2_kac_sites[i].str.split(' ', expand=True)[0]
    try:
        temp_sites = p2_kac_sites[i].loc[bool_mask].str.split('(?<!;)\s', expand=True)[2]
        p2_sites.append(temp_sites)
    except:
        continue

In [48]:
# convert the list into a pandas series     
p1_sites = pd.concat(p1_sites)

# remove the percent confidence and brackets 
p1_sites = p1_sites.str.replace('\(\d{1,6}\)', '').str.replace('\[(.+)\]', '\g<1>')

p1_sites.rename('Acetylated Residues', inplace=True)

p1_sites.head(10)

919           K312
1143          K393
764           K179
550           K334
1145           K52
1142      K49; K52
616     K531; K539
836           K260
946            K60
1093          K226
Name: Acetylated Residues, dtype: object

In [49]:
# convert the list into a pandas series     
p2_sites = pd.concat(p2_sites)

# remove the percent confidence and brackets 
p2_sites = p2_sites.str.replace('\(\d{1,6}\)', '').str.replace('\[(.+)\]', '\g<1>')

p2_sites.rename('Acetylated Residues', inplace=True)

p2_sites.head(10)

752           K52
661          K209
750          K393
609           K60
341          K239
280          K259
812           K46
301    K411; K413
286             K
196          K283
Name: Acetylated Residues, dtype: object

Now, we add the specific sites back to the dataframe and prepare to export

In [51]:
# merge the residues back onto the dataframe
p1 = p1.merge(p1_sites.to_frame(), left_index=True, right_index=True)
p2 = p2.merge(p2_sites.to_frame(), left_index=True, right_index=True)

In [52]:
p1_export_cols = ['Accession', 'Name', 'Symbol', 'Gene Symbol', 'Entrez', 
                  'Matrix', 'IMS', 'Acetylated Residues', 'DKOvsDFC_FC', 'CrATKOvsCrATFL_FC']


p1[p1_export_cols].head(25)

,Accession,Name,Symbol,Gene Symbol,Entrez,Matrix,IMS,Acetylated Residues,DKOvsDFC_FC,CrATKOvsCrATFL_FC
919,Q61425,"Hydroxyacyl-coenzyme A dehydrogenase, mitochon...",Hadh,Hadh,15107,True,False,K312,16.546091,1.302894
1143,Q9CZU6,"Citrate synthase, mitochondrial",Cs,Cs,12974,True,False,K393,16.359824,1.418364
764,Q8K2B3,Succinate dehydrogenase [ubiquinone] flavoprot...,Sdha,Sdha,66945,True,False,K179,16.164221,1.199551
550,Q8BMS1,"Trifunctional enzyme subunit alpha, mitochondrial",Hadha,Hadha,97212,True,False,K334,14.830350,1.352968
1145,Q9CZU6,"Citrate synthase, mitochondrial",Cs,Cs,12974,True,False,K52,14.824442,1.795159
1142,Q9CZU6,"Citrate synthase, mitochondrial",Cs,Cs,12974,True,False,K49; K52,14.108287,1.378787
616,Q03265,"ATP synthase subunit alpha, mitochondrial",Atp5a1,Atp5a1,11946,True,False,K531; K539,13.955865,2.165616
836,Q8QZT1,"Acetyl-CoA acetyltransferase, mitochondrial",Acat1,Acat1,110446,True,False,K260,13.620694,1.284992
946,Q9WUR2,"Enoyl-CoA delta isomerase 2, mitochondrial",Eci2,Eci2,23986,True,False,K60,11.757587,1.287511
1093,Q99LC5,"Electron transfer flavoprotein subunit alpha, ...",Etfa,Etfa,110842,True,False,K226,11.376252,1.483240


In [53]:
p2_export_cols = ['Accession', 'Name', 'Symbol', 'Gene Symbol', 'Entrez', 
                  'Matrix', 'IMS', 'Acetylated Residues', 'DKOvsDFC_FC', 'S3KOvsS3FL_FC']


p2[p2_export_cols].head(25)

,Accession,Name,Symbol,Gene Symbol,Entrez,Matrix,IMS,Acetylated Residues,DKOvsDFC_FC,S3KOvsS3FL_FC
752,Q9CZU6,"Citrate synthase, mitochondrial",Cs,Cs,12974,True,False,K52,203.430666,11.434282
661,Q8BWT1,"3-ketoacyl-CoA thiolase, mitochondrial",Acaa2,Acaa2,52538,True,False,K209,120.487254,8.361982
750,Q9CZU6,"Citrate synthase, mitochondrial",Cs,Cs,12974,True,False,K393,79.247040,12.337543
609,Q9WUR2,"Enoyl-CoA delta isomerase 2, mitochondrial",Eci2,Eci2,23986,True,False,K60,74.078830,11.796914
341,P08249,"Malate dehydrogenase, mitochondrial",Mdh2,Mdh2,17448,True,False,K239,67.106543,13.180614
280,Q8BMS1,"Trifunctional enzyme subunit alpha, mitochondrial",Hadha,Hadha,97212,True,False,K259,57.811284,91.912696
812,P03930,ATP synthase protein 8,Mtatp8,ATP8,17706,False,False,K46,53.756961,10.581788
301,Q8BMS1,"Trifunctional enzyme subunit alpha, mitochondrial",Hadha,Hadha,97212,True,False,K411; K413,50.224529,9.495206
286,Q8BMS1,"Trifunctional enzyme subunit alpha, mitochondrial",Hadha,Hadha,97212,True,False,K,43.299145,8.348417
768,Q9DB20,"ATP synthase subunit O, mitochondrial",Atp5o,Atp5o; LOC100047429; LOC102641678,28080,True,False,K97,41.507593,30.187432


# Export the Data

In [55]:
# write to an excel file
writer = pd.ExcelWriter('processed_files/Williams_2019_PRX_Processed_Data.xlsx', 
                        options={'strings_to_numbers': True})

p1_ro_acetyl.to_excel(writer, sheet_name='p1_DKOvsCrAT_RO_Kac', index=False)
p2_ro_acetyl.to_excel(writer, sheet_name='p2_DKOvsSirt3_RO_Kac', index=False)

p1_acetyl.to_excel(writer, sheet_name='p1_DKOvsCrAT_Kac', index=False)
p2_acetyl.to_excel(writer, sheet_name='p2_DKOvsSirt3_Kac', index=False)

p1_protein.to_excel(writer, sheet_name='p1_DKOvsCrAT_Protein', index=False)
p2_protein.to_excel(writer, sheet_name='p2_DKOvsSirt3_Protein', index=False)

writer.save()

In [56]:
# write to csv files
p1_ro_acetyl.to_csv('processed_files/p1_DKOvsCrAT_RO_Kac.csv', index=False)
p2_ro_acetyl.to_csv('processed_files/p2_DKOvsSirt3_RO_Kac.csv', index=False)

p1_acetyl.to_csv('processed_files/p1_DKOvsCrAT_Kac.csv', index=False)
p2_acetyl.to_csv('processed_files/p2_DKOvsSirt3_Kac.csv', index=False)

p1_protein.to_csv('processed_files/p1_DKOvsCrAT_Protein.csv', index=False)
p2_protein.to_csv('processed_files/p2_DKOvsSirt3_Protein.csv', index=False)

In [57]:
# write to an excel file
writer = pd.ExcelWriter('processed_files/Williams_2019_PRX_Top25.xlsx', 
                        options={'strings_to_numbers': True})

(p1[p1_export_cols].sort_values('DKOvsDFC_FC', ascending=False).head(25)
 .to_excel(writer, sheet_name='p1_Top25_Kac_DKOvsDFC', index=False))

(p1[p1_export_cols].sort_values('CrATKOvsCrATFL_FC', ascending=False).head(25)
 .to_excel(writer, sheet_name='p1_Top25_Kac_CrATKOvsCrATFL', index=False))

(p2[p2_export_cols].sort_values('DKOvsDFC_FC', ascending=False).head(25)
 .to_excel(writer, sheet_name='p2_Top25_Kac_DKOvsDFC', index=False))

(p2[p2_export_cols].sort_values('S3KOvsS3FL_FC', ascending=False).head(25)
 .to_excel(writer, sheet_name='p2_Top25_Kac_S3KOvsS3FL', index=False))

writer.save()

In [58]:
# write to csv files
(p1[p1_export_cols].sort_values('DKOvsDFC_FC', ascending=False).head(25)
 .to_csv('processed_files/p1_Top25_Kac_DKOvsDFC.csv', index=False))

(p1[p1_export_cols].sort_values('CrATKOvsCrATFL_FC', ascending=False).head(25)
 .to_csv('processed_files/p1_Top25_Kac_CrATKOvsCrATFL.csv', index=False))

(p2[p2_export_cols].sort_values('DKOvsDFC_FC', ascending=False).head(25)
 .to_csv('processed_files/p2_Top25_Kac_DKOvsDFC.csv', index=False))

(p2[p2_export_cols].sort_values('S3KOvsS3FL_FC', ascending=False)
 .head(25).to_csv('processed_files/p2_Top25_Kac_S3KOvsS3FL.csv', index=False))